### 01B. Linear Regression with Gradient Descent from Scratch using Python: Clarified

In [ ]:
#Importing all packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp

Generating example dataset

In [ ]:
#Setting random seed
np.random.seed(42)

#Generating age (x) and salaries (y)
x = np.linspace(22,58,10) #Equally spaced 10 data points between age 22 and 58
e = np.random.randn(10) #Random error term
c = 20000 #Intercept I chose this randomly

#Salary based on age of an employee. I chose the equation randomly
y = 10000*x + c + e*80000

Visualizing example dataset

In [ ]:
# Scatter plot between x1 and y
plt.figure(figsize=(8, 5))
plt.scatter(x, y, color='blue', alpha=0.7)

# plt.scatter(np.mean(x), np.mean(y), color='black', marker="h",linewidths=10)
plt.title('Salary vs Age')
plt.xlabel('Age (Years)')
plt.ylabel('Salary (INR)')
plt.ylim(bottom=0)
plt.show()

### 1. Line Rotation Method

a. Creating 10000 lines with different rotational angles and intercepts

In [ ]:
%%time

#Calculate means
xm = np.mean(x)
ym = np.mean(y)

# Scatter plot between x and y
plt.figure(figsize=(8, 5))
plt.scatter(x, y, color='blue', alpha=0.7)

# Draw each line
angles_deg = np.linspace(89.99,45,10000)
angles_rad = np.deg2rad(angles_deg)
intercepts = range(200000, 400000, 10000)
rss = []
slope = []
inter = []

i=1
for a in angles_rad:
    for b in intercepts:
        m = np.tan(a)
        c = b  #Equation to calculate c for the line which needs to be passed through center
        y_rot = m*x + c #y values after 1 single rotation and intercept change
        plt.plot(x, y_rot, color='red', linestyle=':')
        rss_a = np.sum((y-y_rot)**2)
        rss.append(rss_a)
        slope.append(m)
        inter.append(b)

    i = i+1

plt.title('Salary vs Age')
plt.xlabel('Age (Years)')
plt.ylabel('Salary (INR)')

plt.show()

b. Calculating the best slope, intercept and predictions

In [ ]:
#Get the minimum RSS
min_rss = min(rss)

#Get the slope corresponding to the minimum RSS
pos = rss.index(min_rss)
degree = angles_rad[0]
best_slope = np.tan(degree)

#Calculate the best intercept 
int = ym - best_slope*xm

#Calculating prediction values
y_best = best_slope*x + int

print(f"Min RSS: {min_rss},\nBest Slope: {best_slope},\nBest Intercept: {int}")
print(f"Predictions: {y_best}")

c. Visualizing RSS vs Slope from the line rotation method

In [ ]:
# Convert to numpy arrays
slopes = np.array(slope)
intercepts = np.array(inter)
rss_values = np.array(rss)

# Determine shape of the mesh
n_slope = len(set(slope))
n_intercepts = len(set(inter))

# Reshape for surface plot
S = slopes.reshape((n_slope, n_intercepts))
I = intercepts.reshape((n_slope, n_intercepts))
R = rss_values.reshape((n_slope, n_intercepts))

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(S, I, R, cmap='plasma')
fig.colorbar(surf)

ax.set_xlabel('Slope')
ax.set_ylabel('Intercept')
ax.set_zlabel('RSS')
ax.set_title('RSS vs Slope and Intercept')

plt.show()

### 2. Calculating Gradient Descent

a. Calculating Gradient Descent for a generic univariate equation

In [ ]:
# Define symbolic variables
m, c = sp.symbols('m c')
x_i, y_i = sp.symbols('x_i y_i')

# RSS for a single data point (i-th)
RSS_i = (y_i - (m*x_i + c))**2

# Partial derivatives
dRSS_dm = sp.diff(RSS_i, m)
dRSS_dc = sp.diff(RSS_i, c)

print("Partial derivative w.r.t m:")
sp.pprint(dRSS_dm)

print("\nPartial derivative w.r.t c:")
sp.pprint(dRSS_dc)


b. Calculating Gradient Descent for my Salary vs Age equation

In [ ]:
# Define symbolic variables for parameters only
m, c = sp.symbols('m c')
x_i, y_i = sp.symbols('x_i y_i')

# RSS for a single data point
RSS_i = (y_i - (m*x_i + c))**2

# Partial derivatives for one point
dRSS_dm_i = sp.diff(RSS_i, m)
dRSS_dc_i = sp.diff(RSS_i, c)

# Initialize symbolic sums (start at 0)
grad_m_sum = 0
grad_c_sum = 0

# Sum partial derivatives over all data points with numeric xi, yi plugged in. x and y are coming from the top cell where I have defined the salary vs age data.
for xi, yi in zip(x, y):
    grad_m_sum = grad_m_sum + dRSS_dm_i.subs({x_i: xi, y_i: yi})
    grad_c_sum = grad_c_sum + dRSS_dc_i.subs({x_i: xi, y_i: yi})

# Display the symbolic expressions for the total gradients
print("Gradient w.r.t m:")
print(grad_m_sum)

print("\nGradient w.r.t c:")
print(grad_c_sum)

c. Testing the above gradient equation with dummy values (0s)

In [ ]:
# Substitute values of m and c after the loop
mi = 0.0
ci = 0.0

result_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
result_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

print(f"Evaluated gradient at m={mi}, c={ci}:")
print(f"dRSS/dm = {result_m}")
print(f"dRSS/dc = {result_c}")

d. using the gradient equation, creating a loop to iterate over multiple m and c values with a fixed learning rate

In [ ]:
#Manual
eta = 0.01

#Iteration 1
mi=0
ci=0

grad_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
grad_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

print(grad_m)
print(grad_c)

#Iteration 2

m_old = mi
c_old = ci

mnew = m_old-(eta*grad_m)
cnew = c_old-(eta*grad_c)

print("m_new: ",mnew)
print("c_new: ",cnew)

mi = mnew
ci = cnew

grad_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
grad_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

print(grad_m)
print(grad_c)

#Iteration 3

m_old = mi
c_old = ci

mnew = m_old-(eta*grad_m)
cnew = c_old-(eta*grad_c)

print("m_new: ",mnew)
print("c_new: ",cnew)

mi = mnew
ci = cnew

grad_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
grad_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

print(grad_m)
print(grad_c)

In [ ]:
#Hyperparameters
eta = 0.00005 #Leraning rate
mi=5000 #Initial m and c
ci=50000
n_iterations = 1000 #Iterations

for i in range(100000):
    grad_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
    grad_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

    m_old = mi
    c_old = ci

    mnew = m_old-(eta*grad_m)
    cnew = c_old-(eta*grad_c)

    mi = mnew
    ci = cnew

    grad_m = grad_m_sum.subs({m: mi, c: ci}).evalf()
    grad_c = grad_c_sum.subs({m: mi, c: ci}).evalf()

    
    i = i+1
    print(f"Iteration: ",i, "Values of m and c:", mi, ci, "Gradients at {mi} and {ci}:", grad_m, grad_c)

    if (abs(grad_m) < abs(0.001)) & (abs(grad_c) < abs(0.001)):
        break

print(f"Final parameter values for m: {mi} and c: {ci}")
    


e. Visualizing results from OLS and GD methods

In [ ]:
#OLS parameters

m_o = 9861.99256632237
c_o = 61365.18628300569
y_o = m_o*x + c_o

#GD parameters

m_gd = 9861.99256632237
c_gd = 61365.18628300569
y_gd = m_gd*x + c_gd

# Scatter plot between x and y
plt.figure(figsize=(8, 5))
plt.scatter(x, y, color='blue', alpha=0.7)
plt.plot(x, y_o, color='orange', linestyle="-",label='OLS Method')
plt.plot(x, y_gd, color='red', linestyle="-",label='GD Method')

plt.title('Salary vs Age')
plt.xlabel('Age (Years)')
plt.ylabel('Salary (INR)')
plt.legend()

plt.show()

In [ ]:
#Loading California housing dataset
data = datasets.fetch_california_housing()

#Getting a pandas dataframe
df = pd.DataFrame(data['data'])
df['target'] = data['target']
df.columns = data['feature_names']+data['target_names']
df.head()

In [ ]:
#Getting the x and y (regressors/features and target value)
x = np.array(df.drop('MedHouseVal',axis=1))
y = np.array(df['MedHouseVal'])

#Adding intercept to features
X = np.c_[np.ones((x.shape[0], 1)),x]

#Let me start will all beta with 0
beta = np.zeros(X.shape[1])

# Hyperparameters
eta = 0.00000000001 
n_iterations = 100000

for i in range(n_iterations):
    grad = -2 * X.T.dot(y - X.dot(beta))
    beta_new = beta - eta * grad
    print(f"Iteration: ",i, "Values of beta:", beta, "Gradients at beta:", grad)
    if np.all(np.abs(grad) < 0.001):
        break
    else:
        beta = beta_new
    
print(f"Final parameter values for beta: {beta_new}")